### Data tunning

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
datos = pd.read_csv('/content/2.MOCK_DATA.csv')
datos = datos.sort_values(by=['hora']).reset_index(drop=True)

In [ ]:
# Creo la columna 'consumo_del_dia' con datos coherentes
consumo = []
for i in range(1000):
  consumo.append(i/random.randrange(30,38) + i*0.01)
consumo.sort()
consumo = pd.DataFrame(consumo)
consumo = round(consumo,4)
datos = pd.concat([datos, consumo], axis=1)
datos.rename(columns = {0:'consumo_del_dia'}, inplace = True)

In [ ]:
# Adecúo el campo 'hora' para que quede cada 30 segundos (como dice la letra)
datos['fecha2'] = '2022-07-14'
datos = datos.loc[:, datos.columns != 'fecha']
datos['hora2'] = datos['hora'].str[:6]
secs = ['00','30']*500
datos['secs'] = secs
datos['hora_total'] = datos["hora2"] + datos["secs"]
datos = datos.loc[:, datos.columns != 'hora']
datos = datos.loc[:, datos.columns != 'hora2']
datos = datos.loc[:, datos.columns != 'secs']
datos.rename(columns = {'hora_total':'hora'}, inplace = True)
datos['fecha_hora'] = datos["fecha2"] + ' ' + datos['hora']
datos = datos.loc[:, datos.columns != 'fecha2']
datos = datos.loc[:, datos.columns != 'hora']
#datos['fecha_hora'] = pd.to_datetime(datos['fecha_hora'], format='%Y-%m-%d %H:%M')

In [ ]:
# Creo la columna 'bateria_resetante' con los datos de bateria1 y bateria2
bateria_1 = datos['bateria1'].head(500)
bateria_1 = pd.DataFrame(bateria_1).sort_values(by='bateria1', ascending=False).reset_index(drop=True)
bateria_1 = list(bateria_1['bateria1'])
bateria_2 = datos['bateria2'].head(500)
bateria_2 = pd.DataFrame(bateria_2).sort_values(by='bateria2', ascending=False).reset_index(drop=True)
bateria_2 = list(bateria_2['bateria2'])
bateria_restante = bateria_1 + bateria_2
bateria_restante = pd.DataFrame(bateria_restante).reset_index(drop=True)
datos['bateria'] = bateria_restante
datos = datos.loc[:, datos.columns != 'bateria1']
datos = datos.loc[:, datos.columns != 'bateria2']

In [ ]:
# Creo la columna 'dist_supercharger' con los datos de dist_supercharger1 y dist_supercharger2
dist_1 = pd.DataFrame(datos['dist_supercharger1'])
dist_1 = dist_1.sort_values(by='dist_supercharger1', ascending=False)
dist_1 = dist_1.head(500).reset_index(drop=True)
dist_2 = pd.DataFrame(datos['dist_supercharger2'])
dist_2 = dist_2.sort_values(by='dist_supercharger2', ascending=False)
dist_2 = dist_2.head(500).reset_index(drop=True)

dist_1 = list(dist_1['dist_supercharger1'])
dist_2 = list(dist_2['dist_supercharger2'])
dist_supercharger = dist_1 + dist_2
datos['dist_supercharger'] = dist_supercharger
datos = datos.loc[:, datos.columns != 'dist_supercharger1']
datos = datos.loc[:, datos.columns != 'dist_supercharger2']

In [ ]:
# Reordeno las columnas
datos = datos[['id_vehiculo', 'fecha_hora', 'ondas', 'temp_motor', 'vibracion', 'presion',
               'humedad','voltaje', 'velocidad','consumo_del_dia','bateria', 'dist_supercharger']]
datos          

,id_vehiculo,fecha_hora,ondas,temp_motor,vibracion,presion,humedad,voltaje,velocidad,consumo_del_dia,bateria,dist_supercharger
0,24596,2022-07-14 13:00:00,0.749,88.0,0.811,2.42,0.92,345.8,54.4,0.0000,99.8,479.6
1,24596,2022-07-14 13:00:30,0.818,85.1,0.837,3.55,0.79,436.9,71.3,0.0386,99.7,479.6
2,24596,2022-07-14 13:01:00,0.738,98.2,0.761,3.92,0.91,380.0,40.4,0.0741,99.6,479.1
3,24596,2022-07-14 13:01:30,0.894,89.9,1.000,3.71,0.81,402.2,79.9,0.1300,99.4,478.3
4,24596,2022-07-14 13:01:00,0.756,96.2,0.774,3.92,0.95,339.7,52.7,0.1481,99.1,478.2
...,...,...,...,...,...,...,...,...,...,...,...,...
995,24596,2022-07-14 23:55:30,0.802,92.7,0.972,2.16,0.95,417.3,69.3,41.7932,41.0,607.4
996,24596,2022-07-14 23:55:00,0.740,99.0,0.879,2.74,0.86,434.0,59.4,41.9200,40.9,607.3
997,24596,2022-07-14 23:55:30,0.706,88.8,0.928,3.57,0.96,373.7,42.2,42.2933,40.4,606.9
998,24596,2022-07-14 23:57:00,0.963,90.0,0.812,3.90,0.98,392.6,68.3,42.7700,40.3,606.6


### JSON

In [ ]:
datos.to_json("./4.data_to_elastic.json", orient="records", lines=True)

In [ ]:
import json
json.parse("./4.data_to_elastic.json")

AttributeError: ignored

In [ ]:
datos3 = pd.read_json('/content/4.data_to_elastic.json', lines=True)
datos3

,id_vehiculo,fecha_hora,ondas,temp_motor,vibracion,presion,humedad,voltaje,velocidad,consumo_del_dia,bateria,dist_supercharger
0,24596,2022-07-14 13:00:00,0.749,88.0,0.811,2.42,0.92,345.8,54.4,0.0000,99.8,479.6
1,24596,2022-07-14 13:00:30,0.818,85.1,0.837,3.55,0.79,436.9,71.3,0.0386,99.7,479.6
2,24596,2022-07-14 13:01:00,0.738,98.2,0.761,3.92,0.91,380.0,40.4,0.0741,99.6,479.1
3,24596,2022-07-14 13:01:30,0.894,89.9,1.000,3.71,0.81,402.2,79.9,0.1300,99.4,478.3
4,24596,2022-07-14 13:01:00,0.756,96.2,0.774,3.92,0.95,339.7,52.7,0.1481,99.1,478.2
...,...,...,...,...,...,...,...,...,...,...,...,...
995,24596,2022-07-14 23:55:30,0.802,92.7,0.972,2.16,0.95,417.3,69.3,41.7932,41.0,607.4
996,24596,2022-07-14 23:55:00,0.740,99.0,0.879,2.74,0.86,434.0,59.4,41.9200,40.9,607.3
997,24596,2022-07-14 23:55:30,0.706,88.8,0.928,3.57,0.96,373.7,42.2,42.2933,40.4,606.9
998,24596,2022-07-14 23:57:00,0.963,90.0,0.812,3.90,0.98,392.6,68.3,42.7700,40.3,606.6


### CSV

In [ ]:
datos.to_csv('/content/data_to_elastic.csv')